<a href="https://colab.research.google.com/github/IvanLauLinTiong/language-detector/blob/main/xlm_roberta_base_commonlanguage_language_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation and Import Libraries

In [1]:
!pip install datasets transformers[sentencepiece]
!pip install ipywidgets
!pip install sklearn
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

     |████████████████████████████████| 290 kB 4.0 MB/s 
     |████████████████████████████████| 3.1 MB 41.7 MB/s 
     |████████████████████████████████| 1.1 MB 35.3 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 132 kB 46.8 MB/s 
     |████████████████████████████████| 243 kB 48.5 MB/s 
     |████████████████████████████████| 160 kB 46.5 MB/s 
     |████████████████████████████████| 192 kB 53.5 MB/s 
     |████████████████████████████████| 271 kB 50.9 MB/s 
     |████████████████████████████████| 596 kB 49.9 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 3.3 MB 40.2 MB/s 
     |████████████████████████████████| 1.2 MB 34.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# install git lfs 
!sudo apt-get install git-lfs
# !git lfs install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 0s (13.9 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 155222 files and directories cu

In [3]:
# Login your HF account and retrueve HF token
# Note: you must have a HF account created before running this cell 
# and go to next step
from huggingface_hub import notebook_login

notebook_login()


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-crendential store but this isn't the helper defined on your machine.
You will have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal to set it as the default

git config --global credential.helper store


In [ ]:
from huggingface_hub import HfFolder
HF_TOKEN = HfFolder.get_token() 

# Preprocessing

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer

# checkpoint used in preprocessing and modelling
checkpoint = 'xlm-roberta-base'

# dataset used
dataset_name = 'common_language' #'amazon_reviews_multi'

# cache dirs
# CACHE_DATASETS = './cache/datasets/'
# CACHE_PRETRAINED = './cache/pretrained/'


In [ ]:
# load dataset
# train_dataset, test_dataset = load_dataset(dataset_name, 
# #                                            "all_languages", 
#                                            split=['train', 'test'], 
# #                                            cache_dir=CACHE_DATASETS
# #                                            download_mode="force_redownload"
#                                           )
# print(train_dataset)
# print(test_dataset)


dataset = load_dataset(dataset_name)
print(dataset)

Downloading:   0%|          | 0.00/3.76G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset common_language downloaded and prepared to C:\Users\Ivan\.cache\huggingface\datasets\common_language\full\0.1.0\58dfd47953a9a67e82ae6d18c796f4d4f7f9025fa46730bbfcdfeaa5e6d12bf1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'age', 'gender', 'language'],
        num_rows: 22194
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'age', 'gender', 'language'],
        num_rows: 5888
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'age', 'gender', 'language'],
        num_rows: 5963
    })
})


In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'age', 'gender', 'language'],
    num_rows: 22194
})
Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'age', 'gender', 'language'],
    num_rows: 5963
})


In [ ]:
max_input_length = 514

# shuffle dataset
train_dataset = train_dataset.shuffle()
test_dataset = test_dataset.shuffle()

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint,
#                                           cache_dir=CACHE_PRETRAINED
                                         )

# tokenizer helper function
def tokenize(batch):
#     return tokenizer(batch['review_body'], truncation=True, max_length=max_input_length)
    return tokenizer(batch['sentence'], truncation=True, max_length=max_input_length)

# collator
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# set format for pytorch
# train_dataset = train_dataset.remove_columns(['review_body','review_id', 'product_id', 'reviewer_id', 'stars', 'review_title', 'product_category'])
# train_dataset =  train_dataset.rename_column("language", "labels")
# train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
# test_dataset = test_dataset.remove_columns(['review_body','review_id', 'product_id', 'reviewer_id', 'stars', 'review_title', 'product_category'])
# test_dataset =  test_dataset.rename_column("language", "labels")
# test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])


train_dataset = train_dataset.remove_columns(['client_id','path', 'sentence', 'age', 'gender', 'audio'])
train_dataset =  train_dataset.rename_column("language", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.remove_columns(['client_id','path', 'sentence', 'age', 'gender', 'audio'])
test_dataset =  test_dataset.rename_column("language", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print(train_dataset)
print(test_dataset)

Loading cached shuffled indices for dataset at C:\Users\Ivan\.cache\huggingface\datasets\common_language\full\0.1.0\58dfd47953a9a67e82ae6d18c796f4d4f7f9025fa46730bbfcdfeaa5e6d12bf1\cache-99b286d0c1d97efc.arrow


  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 22194
})
Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 5963
})


In [6]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint,
#                                           cache_dir=CACHE_PRETRAINED
                                         )

# tokenizer helper function
def tokenize(batch):
#     return tokenizer(batch['review_body'], truncation=True, max_length=max_input_length)
    return tokenizer(batch['sentence'], truncation=True, max_length=max_input_length)

# collator
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [7]:
# save datasets to datasets folder

training_input_path = f'dataset/train'
train_dataset.save_to_disk(training_input_path)

test_input_path = f'dataset/test'
test_dataset.save_to_disk(test_input_path)

# Training

In [8]:

import numpy as np
from datasets import load_from_disk, load_metric
from transformers import AutoModelForSequenceClassification
from transformers import  Trainer, TrainingArguments
from transformers.trainer_utils import get_last_checkpoint

In [9]:
# hub model id
hub_model_id = f'language-detection-fine-tuned-on-{checkpoint}'

In [11]:
 # load datasets
train_dataset = load_from_disk(training_input_path)
test_dataset = load_from_disk(test_input_path)

print(f"loaded train_dataset length is: {len(train_dataset)}")
print(f"loaded test_dataset length is: {len(test_dataset)}")

# define metrics and metrics function
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Prepare model labels - useful in inference API
labels = train_dataset.features["labels"].names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

2021-11-24 07:10:01,488 - __main__ - INFO -  loaded train_dataset length is: 22194
2021-11-24 07:10:01,491 - __main__ - INFO -  loaded test_dataset length is: 5963


Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [11]:
train_dataset.features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'labels': ClassLabel(num_classes=45, names=['Arabic', 'Basque', 'Breton', 'Catalan', 'Chinese_China', 'Chinese_Hongkong', 'Chinese_Taiwan', 'Chuvash', 'Czech', 'Dhivehi', 'Dutch', 'English', 'Esperanto', 'Estonian', 'French', 'Frisian', 'Georgian', 'German', 'Greek', 'Hakha_Chin', 'Indonesian', 'Interlingua', 'Italian', 'Japanese', 'Kabyle', 'Kinyarwanda', 'Kyrgyz', 'Latvian', 'Maltese', 'Mangolian', 'Persian', 'Polish', 'Portuguese', 'Romanian', 'Romansh_Sursilvan', 'Russian', 'Sakha', 'Slovenian', 'Spanish', 'Swedish', 'Tamil', 'Tatar', 'Turkish', 'Ukranian', 'Welsh'], names_file=None, id=None)}

In [12]:
# download model from model hub
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, 
    num_labels=num_labels, 
    label2id=label2id, 
    id2label=id2label,
#     cache_dir=CACHE_PRETRAINED
)
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [13]:
# define training args
training_args = TrainingArguments(
        hub_model_id,
        evaluation_strategy='epoch',
        save_strategy='epoch',

        num_train_epochs=1,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        warmup_steps=500,
        fp16=True,
        learning_rate=3e-5,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        # push to hub parameters
        push_to_hub=True,
        hub_strategy='every_save',
        hub_model_id=hub_model_id,
        hub_token=HF_TOKEN,
    )


In [14]:
# create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# train model
trainer.train()

# evaluate model
eval_result = trainer.evaluate(eval_dataset=test_dataset)

Cloning https://huggingface.co/ivanlau/language-detection-fine-tuned-on-xlm-roberta-base into local empty directory.


2021-11-24 07:11:02,095 - huggingface_hub.repository - WARNING - Cloning https://huggingface.co/ivanlau/language-detection-fine-tuned-on-xlm-roberta-base into local empty directory.


Using amp fp16 backend
***** Running training *****
  Num examples = 22194
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 22194


Epoch,Training Loss,Validation Loss,Accuracy
1,0.100000,0.188630,0.973839


***** Running Evaluation *****
  Num examples = 5963
  Batch size = 1
Saving model checkpoint to language-detection-fine-tuned-on-xlm-roberta-base/checkpoint-22194
Configuration saved in language-detection-fine-tuned-on-xlm-roberta-base/checkpoint-22194/config.json
Model weights saved in language-detection-fine-tuned-on-xlm-roberta-base/checkpoint-22194/pytorch_model.bin
tokenizer config file saved in language-detection-fine-tuned-on-xlm-roberta-base/checkpoint-22194/tokenizer_config.json
Special tokens file saved in language-detection-fine-tuned-on-xlm-roberta-base/checkpoint-22194/special_tokens_map.json
tokenizer config file saved in language-detection-fine-tuned-on-xlm-roberta-base/tokenizer_config.json
Special tokens file saved in language-detection-fine-tuned-on-xlm-roberta-base/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from language-detection-fine-tuned-on-xlm-roberta-base/checkpoint-22194 (sc

In [15]:
# save best model, metrics and create model card
trainer.create_model_card(model_name=hub_model_id)
trainer.push_to_hub()


Saving model checkpoint to language-detection-fine-tuned-on-xlm-roberta-base
Configuration saved in language-detection-fine-tuned-on-xlm-roberta-base/config.json
Model weights saved in language-detection-fine-tuned-on-xlm-roberta-base/pytorch_model.bin
tokenizer config file saved in language-detection-fine-tuned-on-xlm-roberta-base/tokenizer_config.json
Special tokens file saved in language-detection-fine-tuned-on-xlm-roberta-base/special_tokens_map.json
Several commits (2) will be pushed upstream.


2021-11-24 09:15:13,648 - huggingface_hub.repository - WARNING - Several commits (2) will be pushed upstream.


The progress bars may be unreliable.


2021-11-24 09:15:13,658 - huggingface_hub.repository - WARNING - The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.35k/1.04G [00:00<?, ?B/s]

Upload file runs/Nov24_07-10-47_68a9c49ae68b/events.out.tfevents.1637737867.68a9c49ae68b.73.0:  27%|##6       …

Upload file runs/Nov24_07-10-47_68a9c49ae68b/events.out.tfevents.1637745286.68a9c49ae68b.73.2: 100%|##########…

To https://huggingface.co/ivanlau/language-detection-fine-tuned-on-xlm-roberta-base
   9411137..824c539  main -> main



2021-11-24 09:43:42,417 - huggingface_hub.repository - WARNING - To https://huggingface.co/ivanlau/language-detection-fine-tuned-on-xlm-roberta-base
   9411137..824c539  main -> main



'https://huggingface.co/ivanlau/language-detection-fine-tuned-on-xlm-roberta-base/commit/824c5397db59b4d9ac9f6dc16f70aaa13afd5b1a'

In [16]:
# Saves the model
trainer.save_model('model/')

Saving model checkpoint to model/
Configuration saved in model/config.json
Model weights saved in model/pytorch_model.bin
tokenizer config file saved in model/tokenizer_config.json
Special tokens file saved in model/special_tokens_map.json
